In [1]:
import os
os.chdir('/Users/Digodat/OneDrive - Económicas - UBA/Documentos/Maestría Exactas/EyF')

In [2]:
os.getcwd()

'C:\\Users\\Digodat\\OneDrive - Económicas - UBA\\Documentos\\Maestría Exactas\\EyF'

# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [ ]:
%%bash 
pip install duckdb
pip install jupysql
pip install duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [3]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

In [4]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [5]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('../EyF/datasets/competencia_01_crudo.csv')

,Success


In [6]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('C:/Users/Digodat/OneDrive - Económicas - UBA/Documentos/Maestría Exactas/EyF/datasets/competencia_01_crudo.csv')

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [7]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [8]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_01
group by foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



In [9]:
%%sql
create or replace table clase_ternaria as 

with

clientes_marzo as 
(SELECT
 distinct (numero_de_cliente) as cliente_marzo
 from competencia_01
 where foto_mes=202103),

clientes_abril as 
(SELECT
 distinct (numero_de_cliente) as cliente_abril
 from competencia_01
 where foto_mes=202104),

clientes_mayo as 
(SELECT
 distinct (numero_de_cliente) as cliente_mayo
 from competencia_01
 where foto_mes=202105),

consolidado as 
(select
clientes_marzo.cliente_marzo,
clientes_abril.cliente_abril,
clientes_mayo.cliente_mayo

from clientes_marzo

left join clientes_abril 
on clientes_marzo.cliente_marzo = clientes_abril.cliente_abril

left join clientes_mayo
on clientes_marzo.cliente_marzo = clientes_mayo.cliente_mayo),

conclusion as 
(select 
cliente_marzo as numero_de_cliente,
case
when (cliente_abril is null) then 'baja t+1'
when (cliente_abril is not null) and (cliente_mayo is null) then 'baja t+2'
when (cliente_abril is not null) and (cliente_mayo is not null) then 'continua'
else 'possible error' 
end as clase_ternaria

from consolidado)

select 
*
from conclusion

,Success


In [10]:
%%sql
create or replace table competencia_01_new as 

select
competencia_01.*,
clase_ternaria.clase_ternaria
from competencia_01
left join clase_ternaria
on competencia_01.numero_de_cliente = clase_ternaria.numero_de_cliente

,Success


In [ ]:
%%sql
create or replace table competencia_01_new as 

SELECT
*
from competencia_01_new
where clase_ternaria IS NOT NULL

In [11]:
%%sql
COPY competencia_01_new TO '/Users/Digodat/OneDrive - Económicas - UBA/Documentos/Maestría Exactas/EyF/datasets/competencia_01_new.csv' WITH (FORMAT CSV, HEADER)

,Success


In [ ]:
## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [12]:
%%sql
select 
clase_ternaria,
count(*),
round(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM competencia_01_new),2) AS proportion
from competencia_01_new
group by 1

,clase_ternaria,count_star(),proportion
0,continua,483933,98.55
1,baja t+2,1926,0.39
2,baja t+1,1051,0.21
3,NaN,4153,0.85


## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo